In [219]:
import pandas as pd
import numpy as np
from sklearn import datasets
import sklearn
import tensorflow as tf

In [2]:
tesla = pd.read_csv("tslaMerge.csv", engine="python")

In [3]:
pd.set_option('display.max_columns', None, 'display.max_rows', None)
tesla[283:289]

,date,minute,label,high,low,average,volume,notional,numberOfTrades,marketHigh,marketLow,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime,likes,retweets,neutral,negative,positive,composite
283,2021-01-11,15:59,3:59 PM,811.00,809.66,810.752,5742,4655338.820,84,811.350,809.615,810.676,183128,148457480.8,1131,809.66,810.670,809.880,811.180,-0.037332,-0.042248,1.0,0.0,1.0000,0.000,0.0000,0.0000
284,2021-01-12,9:30,9:30 AM,835.34,827.42,831.042,10922,9076643.385,190,835.400,827.340,831.237,575651,478502403.1,1118,830.94,835.340,831.000,835.000,0.000000,0.000000,2.0,0.0,0.8775,0.033,0.0895,0.3322
285,2021-01-12,9:31,9:31 AM,840.61,835.40,838.040,7824,6556824.030,190,841.570,835.090,838.394,329608,276341442.8,1612,835.53,839.100,835.360,839.220,0.008421,0.008610,3.0,2.0,1.0000,0.000,0.0000,0.0000
286,2021-01-12,9:35,9:35 AM,842.26,839.01,840.756,4396,3695963.045,95,842.398,838.800,840.879,179752,151149728.4,793,841.56,841.875,841.625,841.750,0.011689,0.011600,3.5,0.0,1.0000,0.000,0.0000,0.0000
287,2021-01-12,9:36,9:36 AM,844.25,840.45,841.809,5183,4363095.190,92,844.320,840.430,842.533,229519,193377225.9,1071,841.21,841.920,841.620,841.846,0.012956,0.013589,0.0,0.0,1.0000,0.000,0.0000,0.0000
288,2021-01-12,9:38,9:38 AM,841.59,837.52,839.636,3625,3043681.225,56,841.890,837.210,839.836,173781,145947472.0,777,839.10,841.450,840.120,841.715,0.010341,0.010345,1.0,0.0,0.8860,0.000,0.1140,0.4625


In [4]:
i = 0 #make new column for the target direction to train the model
tesla['direction'] = None
tesla.direction[0] = 0
for i in range(len(tesla)-1):
    if tesla.marketChangeOverTime[i] > 0:
        tesla['direction'][i] = 1
    elif tesla.marketChangeOverTime[i] < 0:
        tesla['direction'][i] = 0
    else:
        if i != 0:
            j = i-1
            change = tesla.marketOpen[i] - tesla.marketClose[j]
            if change >= 0:
                tesla['direction'][i] = 1
            else:
                tesla['direction'][i] = 0
tesla.head()

<ipython-input-4-c39c898b8b91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla.direction[0] = 0
<ipython-input-4-c39c898b8b91>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla['direction'][i] = 1
<ipython-input-4-c39c898b8b91>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tesla['direction'][i] = 0
<ipython-input-4-c39c898b8b91>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,date,minute,label,high,low,average,volume,notional,numberOfTrades,marketHigh,marketLow,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime,likes,retweets,neutral,negative,positive,composite,direction
0,2021-01-11,9:30,9:30 AM,849.820,836.880,842.193,20080,1.691124e+07,395,849.97,836.89,846.436,1152950,975898477.1,2673,849.765,843.990,849.400,843.636,0.000000,0.000000,0.0,0.0,1.0000,0.000,0.0000,0.00000,0
1,2021-01-11,9:31,9:31 AM,850.065,841.820,846.433,8160,6.906892e+06,117,850.33,841.12,847.056,338947,287107193.6,1486,843.870,849.880,843.425,850.028,0.005034,0.000732,0.0,0.0,0.6515,0.237,0.1115,-0.10470,1
2,2021-01-11,9:32,9:32 AM,850.930,844.965,847.940,4343,3.682604e+06,93,851.00,844.42,847.795,233115,197633695.2,1164,850.010,849.660,849.700,848.580,0.006824,0.001606,43.0,0.0,1.0000,0.000,0.0000,0.00000,1
3,2021-01-11,9:33,9:33 AM,852.480,844.000,850.202,17148,1.457926e+07,253,853.23,843.75,848.288,345671,293228476.8,1660,848.475,845.300,848.000,846.550,0.009510,0.002188,0.0,0.0,0.9580,0.000,0.0420,0.26335,1
4,2021-01-11,9:34,9:34 AM,850.150,846.530,847.568,7018,5.948231e+06,112,850.14,845.52,847.983,184584,156524181.1,996,846.570,846.815,846.284,847.304,0.006382,0.001828,3.0,1.0,1.0000,0.000,0.0000,0.00000,1


In [179]:
train = tesla[["likes", "retweets", "numberOfTrades", "neutral", "negative", "positive", "composite"]] #use the sentiment data to train the model
train.head()

,likes,retweets,numberOfTrades,neutral,negative,positive,composite
0,0.0,0.0,395,1.0000,0.000,0.0000,0.00000
1,0.0,0.0,117,0.6515,0.237,0.1115,-0.10470
2,43.0,0.0,93,1.0000,0.000,0.0000,0.00000
3,0.0,0.0,253,0.9580,0.000,0.0420,0.26335
4,3.0,1.0,112,1.0000,0.000,0.0000,0.00000


In [171]:
target = tesla.direction #target data

In [180]:
i=0 #do this for all of the values to make sure that they'll be able to be processed in the model
for i in range(len(train)-1):
    if np.isfinite(train.likes[i]) == False:
        print("likes false index: ", i)
print("\n")
        
i=0 
for i in range(len(train)-1):
    if np.isfinite(train.retweets[i]) == False:
        print("retweets false index: ", i)

i=0 
for i in range(len(train)-1):
    if np.isfinite(train.numberOfTrades[i]) == False:
        print("numberOfTrades false index: ", i)

i=0 
for i in range(len(train)-1):
    if np.isfinite(train.neutral[i]) == False:
        print("neutral false index: ", i)
print("\n")

i=0 
for i in range(len(train)-1):
    if np.isfinite(train.negative[i]) == False:
        print("negative false index: ", i)
print("\n")

i=0 
for i in range(len(train)-1):
    if np.isfinite(train.positive[i]) == False:
        print("positive false index: ", i)
print("\n")

i=0 
for i in range(len(train)-1):
    if np.isfinite(train.composite[i]) == False:
        print("composite false index: ", i)



neutral false index:  5642
neutral false index:  5643


negative false index:  5642
negative false index:  5643


positive false index:  5642
positive false index:  5643


composite false index:  5642
composite false index:  5643


In [182]:
train.neutral[5642] = 1.0
train.neutral[5643] = 1.0
train.negative[5642] = 0.0
train.negative[5643] = 0.0
train.positive[5642] = 0.0
train.positive[5643] = 0.0
train.composite[5642] = 0.0
train.composite[5643] = 0.0
#clean up the NaN 

<ipython-input-182-201acc52e90b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.neutral[5642] = 1.0
/Users/griffinclark/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-182-201acc52e90b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.neutral[5643] = 1.0
<ipython-input-182-201acc5

In [200]:
trainNew = train.values
print(trainNew)

[[ 0.00000000e+00  0.00000000e+00  3.95000000e+02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.17000000e+02 ...  2.37000000e-01
   1.11500000e-01 -1.04700000e-01]
 [ 4.30000000e+01  0.00000000e+00  9.30000000e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 2.20000000e+00  6.66666670e-02  3.40000000e+01 ...  8.66666700e-03
   1.02466667e-01  1.33833333e-01]
 [ 1.25000000e+01  1.26000000e+01  2.30000000e+01 ...  6.04000000e-02
   7.25000000e-02 -1.69500000e-02]
 [ 1.07692308e+00  0.00000000e+00  1.28000000e+02 ...  0.00000000e+00
   1.40230769e-01  2.65161538e-01]]


In [216]:
i=0 #check to make sure all vlaues are int
for i in range(len(target)):
    if type(target[i]) != int:
        print("false: ", i)

targetNew = target.values
targetNew.astype(str).astype(int)
targetNew = np.array(targetNew)
print(targetNew)
print(type(targetNew[0]))

[0 1 1 ... 0 0 0]
<class 'int'>


In [202]:
from sklearn.model_selection import train_test_split

In [226]:
#lb = sklearn.preprocessing.LabelBinarizer()
#targetTest = lb.fit(targetNew)
label_encoder = sklearn.preprocessing.LabelEncoder()
targetNew = label_encoder.fit_transform(targetNew)
sklearn.utils.multiclass.type_of_target(targetNew)

'binary'

In [227]:
X_train, X_test, y_train, y_test = train_test_split(trainNew, targetNew, test_size=0.3, random_state=109)
# 70% training and 30% test
#random state can select records random

In [228]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [242]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5445658424381828


In [243]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))

print("F1 Score:", metrics.f1_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred)))


Precision: 0.5445658424381828
Recall: 1.0
F1 Score: 0.7051377513030529
